In [ ]:
import csv

import led_config_utils
import mesh_config
from funky_lights import wavefront, led_config


all_segments = {}
for config in mesh_config.mesh_configs:
    uid = config.uid_offset + 1
    mesh = wavefront.load_obj(config.mesh)
    segments=[]
    for group in mesh.groups:
        data = led_config_utils.prepare_data(group)
        data_2d = led_config_utils.project_2d(data)
        clusters, labels = led_config_utils.cluster(
            data_2d, eps=config.cluster_eps, min_samples=3)
        points, length = led_config_utils.generate_led_positions(
            data, clusters, labels, start_offset=config.start_offset, end_offset=config.end_offset, leds_per_meter=30)
        print('Segment %s: length=%.1fm, num_clusters=%s, num_leds=%s' %
              (group.name, length, len(clusters), len(points)))
        if len(points) == 0:
            continue
        segment = led_config.Segment(
            uid=uid, name=group.name, points=points, num_leds=points.shape[0], length=length)
        segments.append(segment)
        all_segments[uid] = segment
        # led_config_utils.plot_segment(data, clusters, labels, points)
        uid += 1
    
    csv_header = ['uid', 'name', 'num_leds', 'length', 'reversed']
    csv_data = []
    for segment in segments:
        csv_data.append([segment.uid, 
                        segment.name, 
                        segment.num_leds,
                        '{:.2f}'.format(segment.length),
                        False])

    with open(config.output_csv, 'w', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(csv_header)
        for row in csv_data:
            writer.writerow(row)